#  **PART 2A**: The basics: Gaussian process regression in F3DASM

This step-by-step tutorial with exercises can be followedin order to gain understanding of how regression works in F3DASM.

## 1. Import the necessary packages.

In [ ]:
import f3dasm
import numpy as np
import matplotlib.pyplot as plt
import gpytorch

## 2. Define the hyperparameters

In [ ]:
dimensionality = 1
numsamples = 5

kernel = gpytorch.kernels.ScaleKernel(base_kernel=gpytorch.kernels.RBFKernel())

noise_fix = True
max_retries = 15

## 3. Specify the problem

In [ ]:
fun = f3dasm.functions.Sphere(
    dimensionality=dimensionality,
    scale_bounds=np.tile([0.0, 1.0], (dimensionality, 1)),
    )

Let's plot the function.

In [ ]:
x_plot = np.linspace(0, 1, 500)[:, None] # add 1D plot method for functions!
y_plot = fun(x_plot)

# y_plot = (y_plot - np.mean(y_plot)) / np.std(y_plot)

plt.plot(x_plot, y_plot, 'b--', label='Exact')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

Add the design space, sampler and finally the training data.

In [ ]:
parameter_DesignSpace: f3dasm.DesignSpace = f3dasm.make_nd_continuous_design(
    bounds=np.tile([0.0, 1.0], (dimensionality, 1)),
    dimensionality=dimensionality,
)

sampler = f3dasm.sampling.SobolSequence(design=parameter_DesignSpace, seed=123)

train_data: f3dasm.Data = sampler.get_samples(numsamples=numsamples)
train_data.add_output(output=fun(train_data))

Let's see how the training data looks like.

In [ ]:
train_data.data

In [ ]:
y_unscaled = train_data.get_output_data()
y_scaled = (y_unscaled - np.mean(y_unscaled)) / np.std(y_unscaled)

In [ ]:
# y_unscaled

In [ ]:
# y_scaled

In [ ]:
# train_data.data['output'] = y_scaled

In [ ]:
plt.plot(x_plot, y_plot, 'b--', label='Exact')
plt.scatter(train_data.data['input'], train_data.data['output'], c='b', label='Training data')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

## 4. Regression and prediction

In [ ]:
param = f3dasm.regression.gpr.Sogpr_Parameters(kernel=kernel)

regressor = f3dasm.regression.gpr.Sogpr(
    train_data=train_data, 
    design=train_data.design,
    parameter=param,
    noise_fix=noise_fix,
    # noise_fix=False,
)


In [ ]:
surrogate = regressor.train(optimize=True, max_retries=max_retries)

In [ ]:
# surrogate.model.likelihood.noise

In [ ]:
# import torch

# surrogate.model.covar_module.raw_outputscale = torch.nn.Parameter(torch.tensor(1.))
# surrogate.model.covar_module.base_kernel.raw_lengthscale = torch.nn.Parameter(torch.tensor([[1.]]))

In [ ]:
# cons = gpytorch.constraints.constraints.Interval(1e-5, 1e5)
# surrogate.model.covar_module.base_kernel.register_constraint("raw_lengthscale", cons)
# surrogate.model.covar_module.register_constraint("raw_outputscale", cons)

Let's evaluate the mean and the variance of the Gaussian process posterior.

In [ ]:
x_plot_data = f3dasm.Data(design=train_data.design)
x_plot_data.add_numpy_arrays(input=x_plot, output=x_plot)
pred_mean, pred_var = surrogate.predict(test_input_data=x_plot_data)

ucb, lcb = [pred_mean + 2 * (-1) ** k * np.sqrt(np.abs(pred_var)) for k in range(2)]

Let's see how the prediction looks like.

In [ ]:
plt.plot(x_plot, y_plot, 'b--', label='Exact') # add regression plot
plt.scatter(train_data.data['input'], train_data.data['output'], c='b', label='Training data')
plt.plot(x_plot, pred_mean, color='purple', label='Prediction')
plt.fill_between(x_plot.flatten(), lcb.flatten(), ucb.flatten(), color='purple', alpha=.25, label='Confidence')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
surrogate.model.covar_module.outputscale

In [ ]:
surrogate.model.covar_module.base_kernel.lengthscale

### How does the marginal log likelihood look like?

In [ ]:
import torch
likelihood = surrogate.model.likelihood
model = surrogate.model

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# opt_pars = []
# for _, value in list(model.named_parameters()):
#     opt_par = torch.exp(torch.tensor(value.item()))
#     print(_, opt_par)
#     opt_pars.append(opt_par)

opt_pars = [
    torch.tensor(surrogate.model.likelihood.noise.item()),
    torch.tensor(surrogate.model.covar_module.outputscale.item()),
    torch.tensor(surrogate.model.covar_module.base_kernel.lengthscale.item())
    ]

print(opt_pars)

In [ ]:
noise_level = torch.logspace(torch.log10(opt_pars[0]) - 0.1, torch.log10(opt_pars[0]) + 0.1, steps=30)
amp_scale = torch.logspace(torch.log10(opt_pars[1]) - 0.2, torch.log10(opt_pars[1]) + 0.2, steps=30)
length_scale = torch.logspace(torch.log10(opt_pars[2]) - 0.2, torch.log10(opt_pars[2]) + 0.2, steps=30)

# length_scale_grid, noise_scale_grid = torch.meshgrid(length_scale, noise_level)
length_scale_grid, amp_scale_grid = torch.meshgrid(length_scale, amp_scale)

model.train()
likelihood.train()

mll_plot_list = []
# for scale, noise in zip(length_scale_grid.ravel(), noise_scale_grid.ravel()):
for scale, amp in zip(length_scale_grid.ravel(), amp_scale_grid.ravel()):
    model.covar_module.base_kernel.lengthscale = scale
    # model.likelihood.noise = noise
    model.covar_module.outputscale = amp
    mll_plot_list.append(
        mll(
        surrogate.model(torch.tensor(train_data.get_input_data().to_numpy().flatten())),
        torch.tensor(train_data.get_output_data().to_numpy().flatten())
        )
    )

mll_plot = torch.tensor(mll_plot_list).reshape(length_scale_grid.shape)

model.eval()
likelihood.eval()

plt.figure()
plt.contour(
    length_scale_grid.numpy(),
    # noise_scale_grid.numpy(),
    amp_scale_grid.numpy(),
    -mll_plot.numpy(),
    levels=250,
    # norm=LogNorm(vmin=vmin, vmax=vmax),
)
plt.colorbar()
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Length-scale")
# plt.ylabel("Noise-level")
plt.ylabel("Output-scale")
plt.title("Log-marginal-likelihood")

### Error metrics

How well does this surrogate model perform?

Let us calculate some error metrics, starting with the $L^1$, $L^2$ and $L^{\infty}$ distances. Recall that the $L^p$ distance between two vectors $y,y'\in\mathbb{R}^n$ is defined as
$$\|y-y'\|_{L^p}=\left(\sum_{k=1}^n|y_k-y_k'|^p\right)^{1/p}.$$

In [ ]:
l1_norm = np.linalg.norm(y_plot - pred_mean, ord=1)
l2_norm = np.linalg.norm(y_plot - pred_mean, ord=2)
max_norm = np.linalg.norm(y_plot - pred_mean, ord=np.inf)

l1_norm, l2_norm, max_norm

These numbers are closely related to the mean squared error (MSE) and mean average error (MAE):

In [ ]:
MAE = l1_norm / len(y_plot)
MSE = l2_norm ** 2 / len(y_plot)

MAE, MSE

While these values are commonly used as error metrics, something more robust is needed to apply to objective functions with varying sizes of the output range. This motivates scaling the metrics with the output 

## Comparing GPR results with other packages

### scikit-learn

In [ ]:
import sklearn.gaussian_process

kernel_sklearn = sklearn.gaussian_process.kernels.ConstantKernel() * sklearn.gaussian_process.kernels.RBF()
regressor_sklearn = sklearn.gaussian_process.GaussianProcessRegressor(kernel=kernel_sklearn, n_restarts_optimizer=max_retries,)
surrogate_sklearn = regressor_sklearn.fit(X=train_data.get_input_data(), y=train_data.get_output_data())
mean_sklearn, std_sklearn = surrogate_sklearn.predict(X=x_plot, return_std=True)

ucb_sklearn, lcb_sklearn = [mean_sklearn + 2 * (-1) ** k * std_sklearn for k in range(2)]

plt.plot(x_plot, y_plot, 'b--', label='Exact') # add regression plot
plt.scatter(train_data.data['input'], train_data.data['output'], c='b', label='Training data')
plt.plot(x_plot, mean_sklearn, color='purple', label='Prediction')
plt.fill_between(x_plot.flatten(), lcb_sklearn.flatten(), ucb_sklearn.flatten(), color='purple', alpha=.25, label='Confidence')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
surrogate_sklearn.kernel_.get_params()

### GPy

In [ ]:
import GPy

# kernel_gpy = GPy.kern.RBF()
regressor_gpy = GPy.models.GPRegression(X=train_data.get_input_data(), Y=train_data.get_output_data(), normalizer=True, kernel=None)
regressor_gpy.Gaussian_noise.variance.fix(0)

regressor_gpy.optimize()
# regressor_gpy.optimize_restarts(num_restarts=15)
surrogate_gpy = regressor_gpy

mean_gpy, var_gpy = surrogate_gpy.predict(Xnew=x_plot)

ucb_gpy, lcb_gpy = [mean_gpy + 2 * (-1) ** k * np.sqrt(np.abs(var_gpy)) for k in range(2)]

plt.plot(x_plot, y_plot, 'b--', label='Exact') # add regression plot
plt.scatter(train_data.data['input'], train_data.data['output'], c='b', label='Training data')
plt.plot(x_plot, mean_gpy, color='purple', label='Prediction')
plt.fill_between(x_plot.flatten(), lcb_gpy.flatten(), ucb_gpy.flatten(), color='purple', alpha=.25, label='Confidence')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
surrogate.model.covar_module.base_kernel.lengthscale

In [ ]:
surrogate.model.covar_module.outputscale

In [ ]:
surrogate_sklearn.kernel_.get_params()

In [ ]:
surrogate_gpy

## Exercises
1. Change the cosine GP kernel into an RBF kernel (`gpytorch.kernels.RBFKernel()`). What do you notice?
2. Change the function to regress from the AlpineN2 into the Schwefel function (`f3dasm.functions.Schwefel`). What do you notice?
3. Change the number of data points from `15` into a higher number $\le$`150`. Does the GP regress well?